## Taking a deeper look at the tech products

In [1]:
import pandas as pd
import os
import datetime
import warnings
warnings.filterwarnings('ignore')

### Some cleaning

In [2]:
#Removing NaN rows
big_df = pd.read_csv("./tech.csv")
big_df = big_df.dropna(how="all")



In [3]:
big_df["month_number"] = big_df["Order Date"].str[0:2]

In [4]:
#Removing rows with month number as "Or"
big_df = big_df[big_df["month_number"] != "Or"]


#Converting to month_number to int32 datatype
big_df["month_number"] = big_df["Order Date"].str[0:2]
big_df["month_number"] = big_df["month_number"].astype("int32")

In [5]:
pd.options.display.float_format = "{:.2f}".format

In [55]:
import matplotlib.pyplot as plt

In [109]:
mostPop = big_df['Product'].value_counts()
mostPop = pd.DataFrame(mostPop)
mostPop['Quantity Sold'] = mostPop['Product']
mostPop = mostPop.drop(columns='Product')

## Most Popular Products

In [110]:
mostPop

,Quantity Sold
USB-C Charging Cable,21903
Lightning Charging Cable,21658
AAA Batteries (4-pack),20641
AA Batteries (4-pack),20577
Wired Headphones,18882
Apple Airpods Headphones,15549
Bose SoundSport Headphones,13325
27in FHD Monitor,7507
iPhone,6842
27in 4K Gaming Monitor,6230


## Grouping products bought together

In [75]:
df = big_df[big_df['Order ID'].duplicated(keep=False)]
df['grouped'] = df.groupby('Order ID')['Product'].transform(lambda x:','.join(x))
df = df[['Order ID', 'grouped']].drop_duplicates()
df

,Order ID,grouped
3,176560,"Google Phone,Wired Headphones"
18,176574,"Google Phone,USB-C Charging Cable"
30,176585,"Bose SoundSport Headphones,Bose SoundSport Hea..."
32,176586,"AAA Batteries (4-pack),Google Phone"
119,176672,"Lightning Charging Cable,USB-C Charging Cable"
...,...,...
186781,259296,"Apple Airpods Headphones,Apple Airpods Headphones"
186783,259297,"iPhone,Lightning Charging Cable,Lightning Char..."
186791,259303,"34in Ultrawide Monitor,AA Batteries (4-pack)"
186803,259314,"Wired Headphones,AAA Batteries (4-pack)"


In [76]:
from itertools import combinations
from collections import Counter

count = Counter()

for order in df['grouped']:
    sub_list = order.split(",")
    count.update(Counter(combinations(sub_list, 2)))

print(count.most_common(5))

[(('iPhone', 'Lightning Charging Cable'), 1005), (('Google Phone', 'USB-C Charging Cable'), 987), (('iPhone', 'Wired Headphones'), 447), (('Google Phone', 'Wired Headphones'), 414), (('Vareebadd Phone', 'USB-C Charging Cable'), 361)]


## MBA

This is the percentage of orders that contains the item set. For example, if there are 5 orders in total and {apple,egg} occurs in 3 of them,:

             support{apple,egg} = 3/5 or 60%

The minimum support threshold required by apriori can be set based on knowledge of your domain. In this grocery dataset for example, since there could be thousands of distinct items and an order can contain only a small fraction of these items, setting the support threshold to 0.01% may be reasonable.



confidence
Given two items, A and B, confidence measures the percentage of times that item B is purchased, given that item A was purchased. This is expressed as:

             confidence{A->B} = support{A,B} / support{A}   

Confidence values range from 0 to 1, where 0 indicates that B is never purchased when A is purchased, and 1 indicates that B is always purchased whenever A is purchased. Note that the confidence measure is directional. This means that we can also compute the percentage of times that item A is purchased, given that item B was purchased:

             confidence{B->A} = support{A,B} / support{B}    

In our example, the percentage of times that egg is purchased, given that apple was purchased is:

             confidence{apple->egg} = support{apple,egg} / support{apple}
                                    = (3/5) / (4/5)
                                    = 0.75 or 75%

A confidence value of 0.75 implies that out of all orders that contain apple, 75% of them also contain egg. Now, we look at the confidence measure in the opposite direction (ie: egg->apple):

             confidence{egg->apple} = support{apple,egg} / support{egg}
                                    = (3/5) / (3/5)
                                    = 1 or 100%  

Here we see that all of the orders that contain egg also contain apple. But, does this mean that there is a relationship between these two items, or are they occurring together in the same orders simply by chance? To answer this question, we look at another measure which takes into account the popularity of both items.



lift
Given two items, A and B, lift indicates whether there is a relationship between A and B, or whether the two items are occuring together in the same orders simply by chance (ie: at random). Unlike the confidence metric whose value may vary depending on direction (eg: confidence{A->B} may be different from confidence{B->A}), lift has no direction. This means that the lift{A,B} is always equal to the lift{B,A}:

             lift{A,B} = lift{B,A} = support{A,B} / (support{A} * support{B})   

In our example, we compute lift as follows:

  lift{apple,egg} = lift{egg,apple} = support{apple,egg} / (support{apple} * support{egg})
                  = (3/5) / (4/5 * 3/5) 
                  = 1.25    

One way to understand lift is to think of the denominator as the likelihood that A and B will appear in the same order if there was no relationship between them. In the example above, if apple occurred in 80% of the orders and egg occurred in 60% of the orders, then if there was no relationship between them, we would expect both of them to show up together in the same order 48% of the time (ie: 80% * 60%). The numerator, on the other hand, represents how often apple and egg actually appear together in the same order. In this example, that is 60% of the time. Taking the numerator and dividing it by the denominator, we get to how many more times apple and egg actually appear in the same order, compared to if there was no relationship between them (ie: that they are occurring together simply at random).

In summary, lift can take on the following values:

 * lift = 1 implies no relationship between A and B. 
   (ie: A and B occur together only by chance)

 * lift > 1 implies that there is a positive relationship between A and B.
   (ie:  A and B occur together more often than random)

 * lift < 1 implies that there is a negative relationship between A and B.
   (ie:  A and B occur together less often than random)

In [77]:
counts = list(count.items())

countsFiltered = []

for eachGroup in counts:
    
    repetitions = eachGroup[1]
    
    if repetitions > 3:
        countsFiltered.append(eachGroup)

In [78]:
countsFilteredCol1 = [ x[0] for x in countsFiltered ]
countsFilteredCol2 = [ int(x[1]) for x in countsFiltered ]
dfFiltered = pd.DataFrame(data={"items": countsFilteredCol1, "frequency": countsFilteredCol2})
dfFiltered

,items,frequency
0,"(Google Phone, Wired Headphones)",414
1,"(Google Phone, USB-C Charging Cable)",987
2,"(Bose SoundSport Headphones, Bose SoundSport H...",27
3,"(AAA Batteries (4-pack), Google Phone)",11
4,"(Lightning Charging Cable, USB-C Charging Cable)",58
...,...,...
213,"(Bose SoundSport Headphones, Google Phone)",8
214,"(Bose SoundSport Headphones, Macbook Pro Laptop)",4
215,"(34in Ultrawide Monitor, 27in FHD Monitor)",5
216,"(AAA Batteries (4-pack), Vareebadd Phone)",4


## Calculating support value

In [79]:
total = len(big_df['Order ID'].unique())
dfFiltered['item1'] = dfFiltered['items'].apply(lambda x: x[0])
dfFiltered['item2'] = dfFiltered['items'].apply(lambda x: x[1])
dfFiltered

,items,frequency,item1,item2
0,"(Google Phone, Wired Headphones)",414,Google Phone,Wired Headphones
1,"(Google Phone, USB-C Charging Cable)",987,Google Phone,USB-C Charging Cable
2,"(Bose SoundSport Headphones, Bose SoundSport H...",27,Bose SoundSport Headphones,Bose SoundSport Headphones
3,"(AAA Batteries (4-pack), Google Phone)",11,AAA Batteries (4-pack),Google Phone
4,"(Lightning Charging Cable, USB-C Charging Cable)",58,Lightning Charging Cable,USB-C Charging Cable
...,...,...,...,...
213,"(Bose SoundSport Headphones, Google Phone)",8,Bose SoundSport Headphones,Google Phone
214,"(Bose SoundSport Headphones, Macbook Pro Laptop)",4,Bose SoundSport Headphones,Macbook Pro Laptop
215,"(34in Ultrawide Monitor, 27in FHD Monitor)",5,34in Ultrawide Monitor,27in FHD Monitor
216,"(AAA Batteries (4-pack), Vareebadd Phone)",4,AAA Batteries (4-pack),Vareebadd Phone


In [80]:
total

178437

In [81]:
valueCounts = dfFiltered[['item1']].value_counts()
valueCounts2 = dfFiltered[['item2']].value_counts()
valueCounts.append(valueCounts2)

item1                     
AAA Batteries (4-pack)        17
Apple Airpods Headphones      17
Wired Headphones              17
USB-C Charging Cable          17
Lightning Charging Cable      17
AA Batteries (4-pack)         16
Bose SoundSport Headphones    16
27in 4K Gaming Monitor        13
iPhone                        12
27in FHD Monitor              12
Flatscreen TV                 11
Macbook Pro Laptop            10
34in Ultrawide Monitor        10
Google Phone                   9
ThinkPad Laptop                8
Vareebadd Phone                7
20in Monitor                   7
LG Washing Machine             1
LG Dryer                       1
AA Batteries (4-pack)         18
Apple Airpods Headphones      17
Wired Headphones              17
USB-C Charging Cable          17
AAA Batteries (4-pack)        16
Bose SoundSport Headphones    16
Lightning Charging Cable      16
34in Ultrawide Monitor        14
27in FHD Monitor              12
27in 4K Gaming Monitor        11
Macbook Pro Lapt

In [82]:
valueCounts = big_df[['Product']].value_counts()

In [83]:
pd.reset_option('display.float_format')
dfFiltered['support1'] = dfFiltered['item1'].apply(lambda x: valueCounts[x] / total)
dfFiltered

,items,frequency,item1,item2,support1
0,"(Google Phone, Wired Headphones)",414,Google Phone,Wired Headphones,0.030963
1,"(Google Phone, USB-C Charging Cable)",987,Google Phone,USB-C Charging Cable,0.030963
2,"(Bose SoundSport Headphones, Bose SoundSport H...",27,Bose SoundSport Headphones,Bose SoundSport Headphones,0.074676
3,"(AAA Batteries (4-pack), Google Phone)",11,AAA Batteries (4-pack),Google Phone,0.115677
4,"(Lightning Charging Cable, USB-C Charging Cable)",58,Lightning Charging Cable,USB-C Charging Cable,0.121376
...,...,...,...,...,...
213,"(Bose SoundSport Headphones, Google Phone)",8,Bose SoundSport Headphones,Google Phone,0.074676
214,"(Bose SoundSport Headphones, Macbook Pro Laptop)",4,Bose SoundSport Headphones,Macbook Pro Laptop,0.074676
215,"(34in Ultrawide Monitor, 27in FHD Monitor)",5,34in Ultrawide Monitor,27in FHD Monitor,0.034640
216,"(AAA Batteries (4-pack), Vareebadd Phone)",4,AAA Batteries (4-pack),Vareebadd Phone,0.115677


In [84]:
dfFiltered['support1'] = dfFiltered['item1'].apply(lambda x: valueCounts[x]/total)
dfFiltered

,items,frequency,item1,item2,support1
0,"(Google Phone, Wired Headphones)",414,Google Phone,Wired Headphones,0.030963
1,"(Google Phone, USB-C Charging Cable)",987,Google Phone,USB-C Charging Cable,0.030963
2,"(Bose SoundSport Headphones, Bose SoundSport H...",27,Bose SoundSport Headphones,Bose SoundSport Headphones,0.074676
3,"(AAA Batteries (4-pack), Google Phone)",11,AAA Batteries (4-pack),Google Phone,0.115677
4,"(Lightning Charging Cable, USB-C Charging Cable)",58,Lightning Charging Cable,USB-C Charging Cable,0.121376
...,...,...,...,...,...
213,"(Bose SoundSport Headphones, Google Phone)",8,Bose SoundSport Headphones,Google Phone,0.074676
214,"(Bose SoundSport Headphones, Macbook Pro Laptop)",4,Bose SoundSport Headphones,Macbook Pro Laptop,0.074676
215,"(34in Ultrawide Monitor, 27in FHD Monitor)",5,34in Ultrawide Monitor,27in FHD Monitor,0.034640
216,"(AAA Batteries (4-pack), Vareebadd Phone)",4,AAA Batteries (4-pack),Vareebadd Phone,0.115677


In [85]:
dfFiltered['support2'] = dfFiltered['item2'].apply(lambda x: valueCounts[x]/total)
dfFiltered

,items,frequency,item1,item2,support1,support2
0,"(Google Phone, Wired Headphones)",414,Google Phone,Wired Headphones,0.030963,0.105819
1,"(Google Phone, USB-C Charging Cable)",987,Google Phone,USB-C Charging Cable,0.030963,0.122749
2,"(Bose SoundSport Headphones, Bose SoundSport H...",27,Bose SoundSport Headphones,Bose SoundSport Headphones,0.074676,0.074676
3,"(AAA Batteries (4-pack), Google Phone)",11,AAA Batteries (4-pack),Google Phone,0.115677,0.030963
4,"(Lightning Charging Cable, USB-C Charging Cable)",58,Lightning Charging Cable,USB-C Charging Cable,0.121376,0.122749
...,...,...,...,...,...,...
213,"(Bose SoundSport Headphones, Google Phone)",8,Bose SoundSport Headphones,Google Phone,0.074676,0.030963
214,"(Bose SoundSport Headphones, Macbook Pro Laptop)",4,Bose SoundSport Headphones,Macbook Pro Laptop,0.074676,0.026474
215,"(34in Ultrawide Monitor, 27in FHD Monitor)",5,34in Ultrawide Monitor,27in FHD Monitor,0.034640,0.042071
216,"(AAA Batteries (4-pack), Vareebadd Phone)",4,AAA Batteries (4-pack),Vareebadd Phone,0.115677,0.011573


In [86]:
dfFiltered['supportOfGroup'] = dfFiltered['frequency'].apply(lambda x: x / total)
dfFiltered

,items,frequency,item1,item2,support1,support2,supportOfGroup
0,"(Google Phone, Wired Headphones)",414,Google Phone,Wired Headphones,0.030963,0.105819,0.002320
1,"(Google Phone, USB-C Charging Cable)",987,Google Phone,USB-C Charging Cable,0.030963,0.122749,0.005531
2,"(Bose SoundSport Headphones, Bose SoundSport H...",27,Bose SoundSport Headphones,Bose SoundSport Headphones,0.074676,0.074676,0.000151
3,"(AAA Batteries (4-pack), Google Phone)",11,AAA Batteries (4-pack),Google Phone,0.115677,0.030963,0.000062
4,"(Lightning Charging Cable, USB-C Charging Cable)",58,Lightning Charging Cable,USB-C Charging Cable,0.121376,0.122749,0.000325
...,...,...,...,...,...,...,...
213,"(Bose SoundSport Headphones, Google Phone)",8,Bose SoundSport Headphones,Google Phone,0.074676,0.030963,0.000045
214,"(Bose SoundSport Headphones, Macbook Pro Laptop)",4,Bose SoundSport Headphones,Macbook Pro Laptop,0.074676,0.026474,0.000022
215,"(34in Ultrawide Monitor, 27in FHD Monitor)",5,34in Ultrawide Monitor,27in FHD Monitor,0.034640,0.042071,0.000028
216,"(AAA Batteries (4-pack), Vareebadd Phone)",4,AAA Batteries (4-pack),Vareebadd Phone,0.115677,0.011573,0.000022


In [87]:
dfFiltered['supportProduct'] = dfFiltered['support1'] * dfFiltered['support2']
dfFiltered

,items,frequency,item1,item2,support1,support2,supportOfGroup,supportProduct
0,"(Google Phone, Wired Headphones)",414,Google Phone,Wired Headphones,0.030963,0.105819,0.002320,0.003277
1,"(Google Phone, USB-C Charging Cable)",987,Google Phone,USB-C Charging Cable,0.030963,0.122749,0.005531,0.003801
2,"(Bose SoundSport Headphones, Bose SoundSport H...",27,Bose SoundSport Headphones,Bose SoundSport Headphones,0.074676,0.074676,0.000151,0.005577
3,"(AAA Batteries (4-pack), Google Phone)",11,AAA Batteries (4-pack),Google Phone,0.115677,0.030963,0.000062,0.003582
4,"(Lightning Charging Cable, USB-C Charging Cable)",58,Lightning Charging Cable,USB-C Charging Cable,0.121376,0.122749,0.000325,0.014899
...,...,...,...,...,...,...,...,...
213,"(Bose SoundSport Headphones, Google Phone)",8,Bose SoundSport Headphones,Google Phone,0.074676,0.030963,0.000045,0.002312
214,"(Bose SoundSport Headphones, Macbook Pro Laptop)",4,Bose SoundSport Headphones,Macbook Pro Laptop,0.074676,0.026474,0.000022,0.001977
215,"(34in Ultrawide Monitor, 27in FHD Monitor)",5,34in Ultrawide Monitor,27in FHD Monitor,0.034640,0.042071,0.000028,0.001457
216,"(AAA Batteries (4-pack), Vareebadd Phone)",4,AAA Batteries (4-pack),Vareebadd Phone,0.115677,0.011573,0.000022,0.001339


## Calculating lift

In [88]:
dfFiltered['lift'] = dfFiltered['supportOfGroup'] / dfFiltered['supportProduct']
dfFiltered

,items,frequency,item1,item2,support1,support2,supportOfGroup,supportProduct,lift
0,"(Google Phone, Wired Headphones)",414,Google Phone,Wired Headphones,0.030963,0.105819,0.002320,0.003277,0.708117
1,"(Google Phone, USB-C Charging Cable)",987,Google Phone,USB-C Charging Cable,0.030963,0.122749,0.005531,0.003801,1.455346
2,"(Bose SoundSport Headphones, Bose SoundSport H...",27,Bose SoundSport Headphones,Bose SoundSport Headphones,0.074676,0.074676,0.000151,0.005577,0.027134
3,"(AAA Batteries (4-pack), Google Phone)",11,AAA Batteries (4-pack),Google Phone,0.115677,0.030963,0.000062,0.003582,0.017211
4,"(Lightning Charging Cable, USB-C Charging Cable)",58,Lightning Charging Cable,USB-C Charging Cable,0.121376,0.122749,0.000325,0.014899,0.021817
...,...,...,...,...,...,...,...,...,...
213,"(Bose SoundSport Headphones, Google Phone)",8,Bose SoundSport Headphones,Google Phone,0.074676,0.030963,0.000045,0.002312,0.019390
214,"(Bose SoundSport Headphones, Macbook Pro Laptop)",4,Bose SoundSport Headphones,Macbook Pro Laptop,0.074676,0.026474,0.000022,0.001977,0.011339
215,"(34in Ultrawide Monitor, 27in FHD Monitor)",5,34in Ultrawide Monitor,27in FHD Monitor,0.034640,0.042071,0.000028,0.001457,0.019228
216,"(AAA Batteries (4-pack), Vareebadd Phone)",4,AAA Batteries (4-pack),Vareebadd Phone,0.115677,0.011573,0.000022,0.001339,0.016745


In [89]:
dfFiltered.sort_values('lift', ascending=False)

,items,frequency,item1,item2,support1,support2,supportOfGroup,supportProduct,lift
1,"(Google Phone, USB-C Charging Cable)",987,Google Phone,USB-C Charging Cable,0.030963,0.122749,0.005531,0.003801,1.455346
20,"(Vareebadd Phone, USB-C Charging Cable)",361,Vareebadd Phone,USB-C Charging Cable,0.011573,0.122749,0.002023,0.001421,1.424192
7,"(iPhone, Lightning Charging Cable)",1005,iPhone,Lightning Charging Cable,0.038344,0.121376,0.005632,0.004654,1.210179
0,"(Google Phone, Wired Headphones)",414,Google Phone,Wired Headphones,0.030963,0.105819,0.002320,0.003277,0.708117
43,"(Vareebadd Phone, Wired Headphones)",143,Vareebadd Phone,Wired Headphones,0.011573,0.105819,0.000801,0.001225,0.654414
...,...,...,...,...,...,...,...,...,...
214,"(Bose SoundSport Headphones, Macbook Pro Laptop)",4,Bose SoundSport Headphones,Macbook Pro Laptop,0.074676,0.026474,0.000022,0.001977,0.011339
84,"(Apple Airpods Headphones, 34in Ultrawide Moni...",6,Apple Airpods Headphones,34in Ultrawide Monitor,0.087140,0.034640,0.000034,0.003019,0.011140
102,"(iPhone, Bose SoundSport Headphones)",5,iPhone,Bose SoundSport Headphones,0.038344,0.074676,0.000028,0.002863,0.009786
210,"(ThinkPad Laptop, Wired Headphones)",4,ThinkPad Laptop,Wired Headphones,0.023134,0.105819,0.000022,0.002448,0.009157


In [90]:
lifts = dfFiltered[['item1', 'item2', 'support1', 'support2', 'supportOfGroup', 'lift']]
lifts.sort_values('lift', ascending=False)

,item1,item2,support1,support2,supportOfGroup,lift
1,Google Phone,USB-C Charging Cable,0.030963,0.122749,0.005531,1.455346
20,Vareebadd Phone,USB-C Charging Cable,0.011573,0.122749,0.002023,1.424192
7,iPhone,Lightning Charging Cable,0.038344,0.121376,0.005632,1.210179
0,Google Phone,Wired Headphones,0.030963,0.105819,0.002320,0.708117
43,Vareebadd Phone,Wired Headphones,0.011573,0.105819,0.000801,0.654414
...,...,...,...,...,...,...
214,Bose SoundSport Headphones,Macbook Pro Laptop,0.074676,0.026474,0.000022,0.011339
84,Apple Airpods Headphones,34in Ultrawide Monitor,0.087140,0.034640,0.000034,0.011140
102,iPhone,Bose SoundSport Headphones,0.038344,0.074676,0.000028,0.009786
210,ThinkPad Laptop,Wired Headphones,0.023134,0.105819,0.000022,0.009157


Any lift value that is higher than 1 indicates a strong association between those two products. These values were calulated with the support values which take into account all the products bought together against all transactions and converys whether there is a statistically significant relationship between two products.

Here, Cell phones along with their respective cables indicate a very strong relationship. Customers who buy more than one product along with a smartphone will almost always buy their respective charging cable.

In [91]:
highLift = lifts.loc[lifts.lift > 1]
highLift = highLift[['item1', 'item2', 'lift']]

In [92]:
highLift

,item1,item2,lift
1,Google Phone,USB-C Charging Cable,1.455346
7,iPhone,Lightning Charging Cable,1.210179
20,Vareebadd Phone,USB-C Charging Cable,1.424192
